In [1]:
from config.settings import SRC_DIR
import os
os.chdir(SRC_DIR)

In [ ]:
from document_processing_system.components.data_ingestion import DataIngestion
from document_processing_system.components.data_extraction import DataExtraction

data_ingester = DataIngestion('qc_data/qc_templates/PO 177692 - PO 221421.pdf')
data_extracter = DataExtraction(data_ingester)
pages, tables, images = data_extracter.extract_data()
all_items = data_extracter.filter_pages(pages)
data_extracter.display_data(all_items)

Extracting pages:   0%|          | 0/5 [00:00<?, ?pages/s]

Filtering pages:   0%|          | 0/5 [00:00<?, ?pages/s]


--------Page: 2-----------
Intake Pallet QC Inspection Report
Customer DPS PO 177692 / ISS PO 221421
Supplier Code : SUEXP€
Supplier : Surexport UK Limited
COO : Portugal
Vehicle No : R-4273-BDR Received : 20/01/2025 11:06:15
Vessel : PRIMAFRIO Inspection Date : 20/01/2025 11:29:13 Print date : 20/01/2025
Blackberry 20x150g Punnet
ISS Pallet ID : 8407532 Freshness Technology : N/A
Supplier Pallet ID : 50034008579 Punnet / Pad Type : INFIA K37 40 / BLACK ABSORBENT PAD
Customer Pallet ID : 6197127 Outer : GREEN CARTON
Variety : Sweet Royalla Brand : CORE
Grower : 966321/ Organic? : NO Does Pallet Meet Spec? : YES
GGN : 4049929966321/ PLU? : / 0.00%
Expected Qty 120
RED
Orchard/Farm : N./a End Customer : Tesco
Received Qty 120
Harvest Date : 18/01/2025 DP : 7
Total Defects : 40.09%
Size/Calibre : Packhouse :
Estimated Yield : 80%
Lot Number : 552396 Inspector : Ewa.Jawornicka
Waste : Mould: 11.19%
Minor : Bleeding: 14.92%
Minor : Maroon Druplets: 5.59%
Major : Bleeding: 8.39%
Defects Tot

In [3]:
from document_processing_system.components.model_inference import ModelInference
model = ModelInference()
response = model.process_items(all_items)

Loading model and tokenizer...


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Model loaded successfully!


Processing Items:   0%|          | 0/2 [00:00<?, ?item/s]

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


In [ ]:
import pandas as pd
flat_data = [item for sublist in response for item in sublist]

data = pd.DataFrame(flat_data)
data

,product_name,RAG,expected_qty,received_qty,supplier_code,supplier,coo,received_date,inspection_date,print_date,...,minor_fruit_total,major_defects_tot,major_fruit_total,packs_with_major,box_pack_weights,weight_readings,fruit_weights,qa_comments,packs_fruits_inspected_sample_size,boxes_inspected
0,Blackberry 20x150g Punnet,RED,120,120,SUEXP€,Surexport UK Limited,Portugal,2025-01-20 11:06:15,2025-01-20 11:29:13,2025-01-20,...,110,8.39%,45,0%,Avg 161g ; Min 157g; Max 166g Underweight 0%,157 158 158 159 159 159 160 160 160 160 161 16...,Avg 6g 6,Reject due to mould.,20,1
1,Blackberry 20x150g Punnet,AMBER,117,117,SUEXP€,Surexport UK Limited,Portugal,2025-01-20 11:06:15,2025-01-20 11:29:13,2025-01-20,...,40,5.27%,28,0,Avg 160g ; Min 156g; Max 163g Underweight 0%,156 156 157 157 158 158 159 159 159 160 160 16...,Avg 6g 6,Amber due to mould.,20,1
